In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn
import re
import string
import pandas as pd
import seaborn as sns

In [ ]:
#training data
train_data=pd.read_csv('../input/nlp-getting-started/train.csv')


In [ ]:
train_data.head()

In [ ]:
#drop columns
train_data.drop(['id','keyword','location'],axis=1,inplace=True)

In [ ]:
##null values
train_data.isnull().sum()

## let's do some EDA

In [ ]:
train_data['target'].value_counts().plot.pie()

In [ ]:
sns.countplot(train_data['target'])
plt.show()

In [ ]:
##divide into x and y
X=train_data.drop('target',axis=1)

In [ ]:
y=train_data['target']

In [ ]:
##remove punctuations,stop words
from nltk.stem import WordNetLemmatizer
#ps = PorterStemmer()
wn=WordNetLemmatizer()
corpus=[]
for i in range(0,len(X)):
    review=re.sub('[^a-zA-Z]',' ',X['text'][i])
    review=review.lower()
    review=review.split()
    review=[wn.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review= ' '.join(review)
    corpus.append(review)

In [ ]:
len(corpus),X.shape,len(y)

In [ ]:
corpus[2]

In [ ]:
##train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.2,random_state=42)

In [ ]:
len(X_train),len(y_train)

In [ ]:
##using tokenizer of bert
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
##tokenize function
def tokenize_text(data):
    encoded=tokenizer(data,padding=True,truncation=True,return_tensors='np')
    return encoded.data
        
        

In [ ]:
##tokenize train_data
train_data=tokenize_text(X_train)

In [ ]:

train_data['input_ids'].shape,y_train.shape

In [ ]:
#tokenize test_data
test_data=tokenize_text(X_test)

In [ ]:
test_data['input_ids'].shape,y_test.shape

In [ ]:
##import pretrained model
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

In [ ]:
##training 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss,metrics=['accuracy'])
model.fit(
    train_data,
    np.array(y_train), 
    validation_data=(
        test_data,
        np.array(y_test),
    ),
    batch_size=32,epochs=10
)

In [ ]:
##saving model
model.save_pretrained('bert_79')

In [ ]:
preds=model.predict(test_data)

In [ ]:
classes=np.argmax(preds['logits'],axis=1)

In [ ]:
##accuracy
from sklearn import metrics
metrics.accuracy_score(classes,y_test)

In [ ]:
#confusion matrix
metrics.confusion_matrix(classes,y_test)

In [ ]:
##roc auc
metrics.roc_auc_score(classes,y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(classes,y_test))

In [ ]:
##lets reduce lr during traininig
from tensorflow.keras.optimizers.schedules import PolynomialDecay
batch_size = 32
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs
num_train_steps = (len(train_data)// batch_size) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
    )
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=lr_scheduler)

In [ ]:
import tensorflow as tf
model2 = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
##training
model2.compile(
    optimizer=opt,
    loss=model.compute_loss,
    metrics=['accuracy'],
)
model.fit(train_data,
    np.array(y_train), 
    validation_data=(test_data,
        np.array(y_test),
    ),
    batch_size=32,epochs=10
)

In [ ]:
##save
model2.save_pretrained('bert_80')

In [ ]:
preds=model.predict(test_data)

In [ ]:
classes=np.argmax(preds['logits'],axis=1)

In [ ]:
##confusion matrix
metrics.confusion_matrix(classes,y_test)

In [ ]:
metrics.roc_auc_score(classes,y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(classes,y_test))

In [ ]:
##making predictions
pred_data=pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
pred_data.head()

In [ ]:
pred_data.drop(['id','keyword','location'],axis=1,inplace=True)

In [ ]:
pred_data.columns

In [ ]:
##null values
pred_data.isnull().sum()

In [ ]:
##remove stop words and punctuations
from nltk.stem import WordNetLemmatizer
#ps = PorterStemmer()
wn=WordNetLemmatizer()
corpus=[]
for i in range(0,len(pred_data)):
    review=re.sub('[^a-zA-Z]',' ',pred_data['text'][i])
    review=review.lower()
    review=review.split()
    review=[wn.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review= ' '.join(review)
    corpus.append(review)

In [ ]:
len(corpus)

In [ ]:
pred_data=tokenize_text(corpus)

In [ ]:
pred_data['input_ids'].shape

In [ ]:
##predictions
preds=model.predict(pred_data)

In [ ]:
classes=np.argmax(preds['logits'],axis=1)
classes

In [ ]:
##submission file
submisssions=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
submisssions.head()

In [ ]:
##pass our predicted values
submisssions['target']=classes

In [ ]:
submisssions['target'].value_counts()

In [ ]:
##csv file
submisssions.to_csv('submission1.csv',index = False);

<a href="./submission1.csv"> Download File </a>